In [ ]:
import sys
import numpy as np
import dolfin as d
from matplotlib import pyplot as plt
sys.path.append("/root/shared/gitrepos/smart-nanopillars/utils")
from smart_analysis import load_solution
indentation = 2.8
a0_NPC = 5.0
mesh_parent_folder = "/root/shared/gitrepos/smart-comp-sci-data/meshes/nanopillars_finalCalcCoarse"
mesh_folder = f"nanopillars_indent{indentation}"
results_parent_folder = "/root/scratch/nanopillar-sims/results_nanopillars_indentation_REDO5sens"
results_folder = f"nanopillars_indent{indentation}_a0_{a0_NPC}"
tVec = np.loadtxt(f"{results_parent_folder}/{results_folder}/tvec.txt")
stress_fiber_int = []
kNPC_int = []
if indentation > 0 and a0_NPC > 0:
    for sol in load_solution(mesh_file=f"{mesh_parent_folder}/{mesh_folder}/spreadCell_mesh.h5", 
                                results_file=[f"{results_parent_folder}/{results_folder}/FActin.h5", 
                                            f"{results_parent_folder}/{results_folder}/Myo_A.h5",
                                            f"{results_parent_folder}/{results_folder}/NPC_A.h5",
                                            f"{results_parent_folder}/{results_folder}/aNPC.h5"],
                                idx=0):
        FActin, Myo_A, NPC_A, aNPC = sol[:]
        dxCyto = d.Measure("dx", FActin.function_space().mesh())
        dxNE = d.Measure("dx", NPC_A.function_space().mesh())
        volCyto = d.assemble(1.0*dxCyto)
        stress_fiber_int.append(d.assemble(FActin*Myo_A*dxCyto)/volCyto)
        kinb = 1.0
        kin = 10.0
        kNPC_int.append(d.assemble(d.exp((aNPC-1)/a0_NPC)*(kinb + kin*NPC_A)*dxNE))
        print(len(kNPC_int))
else:
    for sol in load_solution(mesh_file=f"{mesh_parent_folder}/{mesh_folder}/spreadCell_mesh.h5", 
                                results_file=[f"{results_parent_folder}/{results_folder}/FActin.h5", 
                                            f"{results_parent_folder}/{results_folder}/Myo_A.h5",
                                            f"{results_parent_folder}/{results_folder}/NPC_A.h5"],
                                idx=0):
        FActin, Myo_A, NPC_A = sol[:]
        dxCyto = d.Measure("dx", FActin.function_space().mesh())
        dxNE = d.Measure("dx", NPC_A.function_space().mesh())
        volCyto = d.assemble(1.0*dxCyto)
        stress_fiber_int.append(d.assemble(FActin*Myo_A*dxCyto)/volCyto)
        kinb = 1.0
        kin = 10.0
        kNPC_int.append(d.assemble((kinb + kin*NPC_A)*dxNE))
        print(len(kNPC_int))


stress_fiber_int = np.array(stress_fiber_int)
NPC_int = np.array(kNPC_int)
fig, ax = plt.subplots(2,1)
ax[0].plot(tVec, stress_fiber_int)
ax[1].plot(tVec, NPC_int)

In [ ]:
import matplotlib as mpl
params = {'axes.labelsize': 12,
            'axes.titlesize': 12,
            'legend.fontsize': 9,
            'xtick.labelsize':10,
            'ytick.labelsize': 10,
            'figure.figsize': (5,3),
            'pdf.fonttype': 42,
            'ps.fonttype': 42,
            'axes.spines.top': False,
            'axes.spines.right': False,
            'legend.loc': "best"}
mpl.rcParams.update(params)
from scipy.integrate import odeint
from scipy import optimize
volCyto = d.assemble(1.0*dxCyto)
volNuc = 68.6
SANuc = d.assemble(1.0*dxNE)
geomParam = [volCyto, volNuc, SANuc, stress_fiber_int, NPC_int]

def mechanotransduction_ode_calc(timeSpan, geomParam, ruptureParam):
    # # input:
    # #     timeSpan is a vector of start and stop times (e.g. timeSpan = [0 10.0])
    # #
    # # output:
    # #     sol gives the solution computed by odeint

    Yinit = [0.2, 0.7, 0.0]
    # ode rate
    def f(Y, t, geomParam, ruptureParam):
        volCyto, volNuc, SANuc, stress_fiber_int, NPC_int =  geomParam[:]
        krupture, upsilon = ruptureParam[:]
        kNC = 0.14
        kCN = 0.56
        kCY = 7.6e-4*np.interp(t, tVec, stress_fiber_int)
        Nconvert = 602.2
        # kinb = 1.0
        # kin = 10.0
        # kin_NPC = kinb*SANuc + kin*np.interp(t, tVec, NPC_int)
        kin_NPC = np.interp(t, tVec, kNPC_int)
        kout_NPC = 1.0
        Ytot = 0.9*volCyto*Nconvert + 0.7*volNuc*Nconvert

        Y0 = (Ytot - Nconvert*(Y[0]*volCyto + (Y[1]+Y[2])*volNuc))/(Nconvert*volCyto)
        if Y0 < 0:
            print(Y0)
        dY = [kNC*Y0 - (kCN+kCY)*Y[0] + (krupture/(Nconvert*volNuc))*(Y[2] - upsilon*Y[0]),
              (1/(Nconvert*volNuc))*((kin_NPC + krupture*upsilon)*Y0 - (SANuc*kout_NPC + krupture)*Y[1]),
              (krupture/(Nconvert*volNuc))*(upsilon*Y[0] - Y[2])]
        return dY

    t = np.linspace(timeSpan[0], timeSpan[1], 1000)
    sol = odeint(f, Yinit, t, args=(geomParam,ruptureParam))
    return t, sol

def mechanotransduction_root_calc(geomParam, ruptureParam):

    Yinit = [0.2, 0.7, 0.0]
    volCyto, volNuc, SANuc, stress_fiber_int, NPC_int =  geomParam[:]
    krupture, upsilon = ruptureParam[:]
    # ode rate
    def f_res(Y):
        kNC = 0.14
        kCN = 0.56
        kCY = 7.6e-4*np.interp(1e4, tVec, stress_fiber_int)
        Nconvert = 602.2
        kin_NPC = np.interp(1e4, tVec, kNPC_int)
        kout_NPC = 1.0
        Ytot = 0.9*volCyto*Nconvert + 0.7*volNuc*Nconvert
        Y0 = (Ytot - Nconvert*(Y[0]*volCyto + (Y[1]+Y[2])*volNuc))/(Nconvert*volCyto)
        dY = [kNC*Y0 - (kCN+kCY)*Y[0] + (krupture/(Nconvert*volNuc))*(Y[2] - upsilon*Y[0]),
              (1/(Nconvert*volNuc))*((kin_NPC + krupture*upsilon)*Y0 - (SANuc*kout_NPC + krupture)*Y[1]),
              (krupture/(Nconvert*volNuc))*(upsilon*Y[0] - Y[2])]
        return dY

    sol = optimize.root(f_res, x0=Yinit)
    return sol

ode_sol = mechanotransduction_ode_calc([0, 100000], geomParam, [0.0, 0.0])
Yphos = ode_sol[1][:,0]
Ynuc = ode_sol[1][:,1]
Ynucphos = ode_sol[1][:,2]
Nconvert = 602.2
Ytot = 0.9*volCyto*Nconvert + 0.7*volNuc*Nconvert
Ydephos = (Ytot - Nconvert*(Yphos*volCyto + (Ynuc+Ynucphos)*volNuc))/(Nconvert*volCyto)
YRatioCur = (Ynuc+Ynucphos)/(Ydephos+Yphos)
YAPnopore = YRatioCur[-1]

krupture0 = np.logspace(1,3,21)
upsilon0 = np.logspace(-1,1,21)
krupture, upsilon = np.meshgrid(krupture0, upsilon0)
YAPratio = np.zeros_like(krupture)
tFinal = 3600
for i in range(len(krupture[:,0])):
    for j in range(len(krupture[0,:])):
        ode_sol = mechanotransduction_ode_calc([0, tFinal], geomParam, [krupture[i,j], upsilon[i,j]])
        Yphos = ode_sol[1][:,0]
        Ynuc = ode_sol[1][:,1]
        Ynucphos = ode_sol[1][:,2]
        Ydephos = (Ytot - Nconvert*(Yphos*volCyto + (Ynuc+Ynucphos)*volNuc))/(Nconvert*volCyto)
        YRatioCur = (Ynuc+Ynucphos)/(Ydephos+Yphos)
        # plt.plot(ode_sol[0]/3600, YRatioCur)
        YAPratio[i,j] = YRatioCur[-1]
        # root_sol = mechanotransduction_root_calc(geomParam, [krupture[i,j], upsilon[i,j]])
        # Yphos, Ynuc, Ynucphos = root_sol.x
        # Ydephos = (Ytot - Nconvert*(Yphos*volCyto + (Ynuc+Ynucphos)*volNuc))/(Nconvert*volCyto)
        # YRatioCur = (Ynuc+Ynucphos)/(Ydephos+Yphos)
        # YAPratio[i,j] = YRatioCur
        # plt.plot(3, YAPratio[i,j], 'o')

        print(f"{i},{j}")

pore_dir = f"/root/shared/gitrepos/smart-nanopillars/analysis_data/npy-files-nanopillars-withwasp-pores"
r0 = np.array([0.1, 0.2])
k0 = np.array([10000, 10000])
krupture0Sim = k0*np.pi*r0**2 / 8
upsilon0Sim = [1.0, 5.0]
kruptureSim, upsilonSim = np.meshgrid(krupture0Sim, upsilon0Sim)
rMat, upsilon1 = np.meshgrid(r0, upsilon0Sim)
kMat, upsilon2 = np.meshgrid(k0, upsilon0Sim)
YAPratioSim = np.zeros_like(kruptureSim)

for i in range(len(kruptureSim[:,0])):
    for j in range(len(kruptureSim[0,:])):
        var_names_saved = ["YAPTAZ", "YAPTAZ_nuc", "YAPTAZ_nuc_phos", "YAPTAZ_phos"]
        plot_names = ["YAPTAZ_phos", "YAPTAZ", "YAPTAZ_nuc", "YAPTAZ_nuc_phos"]

        cur_file = (f"nanopillars_indent{indentation}_pore_size{rMat[i,j]}_"
                f"loc0.0_rate1_transport{kMat[i,j]}_"
                f"ratio{upsilonSim[i,j]:.0f}_results_all.npy")
        cur_file = f"{pore_dir}/{cur_file}"
        try:
            results_cur = np.load(cur_file)
            tSim, YAPnp, YAPnuc, YAPphos_nuc, YAPphos = results_cur[:]
            YAPratioSimCur = (YAPnuc + YAPphos_nuc) / (YAPphos + YAPnp)
            # plt.plot(tSim/3600, YAPratioSimCur, '--')
            YAPratioSim[i,j] = np.interp(tFinal, tSim, YAPratioSimCur)
        except:
            print("could not load")

plt.xlim([0, 1])
plt.xlabel('Time (h)')
plt.ylabel('YAP/TAZ N/C')
# plt.ylim([2.8, 3.8])

# plt.figure()
# for i in range(len(krupture0)):
#     plt.plot(upsilon[:,i], YAPratio[:,i])
#     plt.plot(upsilon[:,i], YAPratioSim[:,i], '--')
# plt.savefig("200nm_10000.pdf", format="pdf")

In [ ]:
from matplotlib.colors import ListedColormap
plt.figure()
cmap_cur = plt.get_cmap('coolwarm')
cmap_big = mpl.colormaps['coolwarm']
upper_lower_ratio = 0.5
clist1 = cmap_big(np.linspace(0, 0.5, 128))
clist2 = cmap_big(np.linspace(0.5, 1.0, int(upper_lower_ratio*128)))
newcmp = ListedColormap(np.concatenate((clist1, clist2[1:])))
logFoldVals = np.linspace(-1, 1*upper_lower_ratio, len(newcmp.colors))
cVals = np.linspace(0, 1, len(newcmp.colors))

ax = plt.axes()
logFoldChange = np.log10(YAPratio / YAPnopore)
im = plt.imshow(logFoldChange.transpose(), cmap = newcmp)
plt.colorbar(im)
plt.ylim([-0.5,len(krupture0)-0.5])
plt.clim([-1, 1*upper_lower_ratio])
xvals = np.arange(0,len(upsilon0),1)
xlabels = []
for i in range(len(xvals)):
    xlabels.append(f"{upsilon0[i]:.1f}")
plt.xlabel('Entry/exit ratio')
yvals = np.arange(0,len(krupture0),1)
ylabels = []
for i in range(len(yvals)):
    ylabels.append(f"{krupture0[i]:.1f}")
plt.ylabel('Transport rate')
plt.xticks(xvals[0::2], xlabels[0::2])
plt.yticks(yvals[0::2], ylabels[0::2])

logFoldChangeSim = np.log10(YAPratioSim / YAPnopore)
for i in range(len(kruptureSim[:,0])):
    for j in range(len(kruptureSim[0,:])):
        upsilonScaled = np.interp(np.log(upsilonSim[i,j]), np.log(upsilon0), xvals)
        kruptureScaled = np.interp(np.log(kruptureSim[i,j]), np.log(krupture0), yvals)
        colorVal = np.interp(logFoldChangeSim[i,j], logFoldVals, cVals)
        plt.plot(upsilonScaled, kruptureScaled, 'o', color=newcmp(colorVal), mec='k')
# plt.savefig("pore_phase_diagram_newColorWithPoints.pdf", format="pdf")